<a href="https://colab.research.google.com/github/PashaIanko/Kaggle.Titanic/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
a = 2

In [ ]:
a


2

In [ ]:
b = 3

In [ ]:
b

3